In [1]:
import xarray as xr
# import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
# import dask.array as da

In [2]:
def rmse(mod,obs):
    mod = mod.assign_coords(time=obs.time)
    rmse = ((((mod - obs)**2).mean(dim = 'time')**0.5)).rename('rmse')
    return rmse


In [3]:
def kge(mod,obs,obsclim,obsstd):
    #taslut.assign_coords(time=obs.time)
    mod = mod.assign_coords(time=obs.time)
   
    modclim = mod.mean(dim = 'time')
    modstd = mod.std(dim = 'time')
    

    B = modclim/obsclim
    G = (modstd/modclim)/(obsstd/ obsclim)
    P = xr.corr(mod , obs, dim= 'time')
    kge = (1-((P-1)**2 + (B - 1)**2 + (G-1)**2)**(0.5)).rename('kge')
    return kge

In [4]:
realisation = 'r1i1p1f1'
model = 'cesm2'

tas = xr.open_dataset('CMIP/tas_'+ model + '_' + realisation + '_obs.nc')
mnlut=xr.open_dataset('CMIP/mnLut_'+ model + '_' + realisation +'_obs.nc')
mpd=xr.open_dataset('review_files/mapped_tas_'+ model + '_new.nc')

mnlut_cor=xr.open_dataset('review_files/mnlut_tascor_'+ model +'_2new.nc')
mpd_cor=xr.open_dataset('review_files/mapped_tascor_'+ model +'_2new.nc')

obs = xr.open_dataset('CMIP/Tair_merge.nc')

obscor = obs.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))



In [6]:
obs_std = obs.Tair.std(dim = 'time')
obs_clim = obs.Tair.mean(dim = 'time')
obscor_std = obscor.Tair.std(dim = 'time')
obscor_clim = obscor.Tair.mean(dim = 'time')

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [7]:
mnlut_kge = kge(mnlut.tasLut,obs.Tair,obs_clim,obs_std)
mnlut_rms = rmse(mnlut.tasLut,obs.Tair)
mnlut_cor_kge = kge(mnlut_cor.tasLut,obscor.Tair,obscor_clim,obscor_std)
mnlut_cor_rms = rmse(mnlut_cor.tasLut,obscor.Tair)


/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [7]:
mpd_kge = kge(mpd.tas,obs.Tair,obs_clim,obs_std)
mpd_rms = rmse(mpd.tas,obs.Tair)
mpd_cor_kge = kge(mpd_cor.tas,obscor.Tair,obscor_clim,obscor_std)
mpd_cor_rms = rmse(mpd_cor.tas,obscor.Tair)

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs

In [8]:
dif_rms = mpd_rms - mnlut_rms
dif_kge = mpd_kge - mnlut_kge
difcor_rms = mpd_cor_rms - mnlut_cor_rms
difcor_kge = mpd_cor_kge - mnlut_cor_kge

In [9]:
dif_rms = dif_rms.to_dataset()
dif_kge = dif_kge.to_dataset()
difcor_rms = difcor_rms.to_dataset()
difcor_kge = difcor_kge.to_dataset()

In [10]:
dif_rms.to_netcdf('review_files/rmse_difmap_'+model+'.nc')
dif_kge.to_netcdf('review_files/kge_difmap_'+model+'.nc')
difcor_rms.to_netcdf('review_files/rmse_difmap_cor_'+model+'.nc')
difcor_kge.to_netcdf('review_files/kge_difmap_cor_'+model+'.nc')

In [9]:
mnlut_rms = mnlut_rms.to_dataset()
mnlut_kge=mnlut_kge.to_dataset()
mnlut_cor_rms = mnlut_cor_rms.to_dataset()
mnlut_cor_kge=mnlut_cor_kge.to_dataset()

In [17]:
mpd_rms = mpd_rms.to_dataset()
mpd_kge=mpd_kge.to_dataset()
mpd_cor_rms = mpd_cor_rms.to_dataset()
mpd_cor_kge=mpd_cor_kge.to_dataset()

In [11]:
mnlut_rms.to_netcdf('review_files/mnlut_rms_'+model+'_2new.nc')
mnlut_kge.to_netcdf('review_files/mnlut_kge_'+model+'_2new.nc')
mnlut_cor_rms.to_netcdf('review_files/mnlut_cor_rms_'+model+'_2new.nc')
mnlut_cor_kge.to_netcdf('review_files/mnlut_cor_kge_'+model+'_2new.nc')

In [19]:
mpd_rms.to_netcdf('review_files/mpd_rms_'+model+'_2new.nc')
mpd_kge.to_netcdf('review_files/mpd_kge_'+model+'_2new.nc')
mpd_cor_rms.to_netcdf('review_files/mpd_cor_rms_'+model+'_2new.nc')
mpd_cor_kge.to_netcdf('review_files/mpd_cor_kge_'+model+'_2new.nc')

### Repeat for UKESM

In [32]:
realisation = 'r1i1p1f2'
model = 'ukesm'

tas = xr.open_dataset('CMIP/tas_'+ model + '_' + realisation + '_obs.nc')
mnlut=xr.open_dataset('CMIP/mnLut_'+ model + '_' + realisation +'_obs.nc')
mpd=xr.open_dataset('review_files/mapped_tas_'+ model + '.nc')

mnlut_cor=xr.open_dataset('review_files/mnlut_tascor_'+ model +'.nc')
mpd_cor=xr.open_dataset('review_files/mapped_tascor_'+ model +'.nc')

obs = xr.open_dataset('CMIP/Tair_merge.nc')

In [21]:
obs_std = obs.Tair.std(dim = 'time')
obs_clim = obs.Tair.mean(dim = 'time')

In [33]:
mnlut = mnlut.assign_coords(time=obs.time)
mnlut_cor = mnlut_cor.assign_coords(time=obs.time)

mpd = mpd.assign_coords(time=obs.time)
mpd_cor = mpd_cor.assign_coords(time=obs.time)


In [36]:
mnlut_kge = kge(mnlut.tasLut,mnlut_clim.tasLut,mnlut_std.tasLut,obs.Tair,obs_clim.Tair,obs_std.Tair)
mnlut_rms = rmse(mnlut.tasLut,obs.Tair)
mnlut_cor_kge = kge(mnlut_cor.tasLut,mnlut_cor_clim.tasLut,mnlut_cor_std.tasLut,obs.Tair,obs_clim,obs_std)
mnlut_cor_rms = rmse(mnlut_cor.tasLut,obs.Tair)

In [28]:
mpd_kge = kge(mpd.tas,mpd_clim.tas,mpd_std.tas,obs.Tair,obs_clim,obs_std)
mpd_rms = rmse(mpd.tas,obs.Tair)
mpd_cor_kge = kge(mpd_cor.tas,mpd_cor_clim.tas,mpd_cor_std.tas,obs.Tair,obs_clim,obs_std)
mpd_cor_rms = rmse(mpd_cor.tas,obs.Tair)

In [37]:
mnlut_rms = mnlut_rms.to_dataset()
mnlut_kge=mnlut_kge.to_dataset()
mnlut_cor_rms = mnlut_cor_rms.to_dataset()
mnlut_cor_kge=mnlut_cor_kge.to_dataset()

In [29]:
mpd_rms = mpd_rms.to_dataset()
mpd_kge=mpd_kge.to_dataset()
mpd_cor_rms = mpd_cor_rms.to_dataset()
mpd_cor_kge=mpd_cor_kge.to_dataset()

In [38]:
mnlut_rms.to_netcdf('review_files/mnlut_rms_'+model+'.nc')
mnlut_kge.to_netcdf('review_files/mnlut_kge_'+model+'.nc')
mnlut_cor_rms.to_netcdf('review_files/mnlut_cor_rms_'+model+'.nc')
mnlut_cor_kge.to_netcdf('review_files/mnlut_cor_kge_'+model+'.nc')

In [31]:
mpd_rms.to_netcdf('review_files/mpd_rms_'+model+'.nc')
mpd_kge.to_netcdf('review_files/mpd_kge_'+model+'.nc')
mpd_cor_rms.to_netcdf('review_files/mpd_cor_rms_'+model+'.nc')
mpd_cor_kge.to_netcdf('review_files/mpd_cor_kge_'+model+'.nc')

### FOR LST

In [7]:
model = 'cesm2'

tas = xr.open_dataset('lstmip/tslsi_'+ model + '_obs.nc')

mnlut=xr.open_dataset('lstmip/mnLut_'+ model  +'_obs.nc')
mpd=xr.open_dataset('review_files/mapped_tslsi_'+ model + '_new.nc')

mnlut_cor=xr.open_dataset('review_files/mnlut_tslsicor_'+ model +'_2new.nc')
mpd_cor=xr.open_dataset('review_files/mapped_tslsicor_'+ model +'_2new.nc')

obs = xr.open_dataset('lstmip/modlst.nc')
obs = obs.sel(time = slice(obs.time[23],obs.time[166]))
obscor = obs.sel(lat = slice(83.4,-55.15),lon = slice(-180.0,178.75))

In [8]:
obs_std = obs.std(dim = 'time')
obs_clim = obs.mean(dim = 'time')
obscor_std = obscor.std(dim = 'time')
obscor_clim = obscor.mean(dim = 'time')

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [9]:
mnlut_kge = kge(mnlut.tslsiLut,obs.lst,obs_clim.lst,obs_std.lst)
mnlut_rms = rmse(mnlut.tslsiLut,obs.lst)
mnlut_cor_kge = kge(mnlut_cor.tslsiLut,obscor.lst,obscor_clim.lst,obscor_std.lst)
mnlut_cor_rms = rmse(mnlut_cor.tslsiLut,obscor.lst)

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [10]:
mpd_kge = kge(mpd.tas,obs.lst,obs_clim.lst,obs_std.lst)
mpd_rms = rmse(mpd.tas,obs.lst)
mpd_cor_kge = kge(mpd_cor.tas,obscor.lst,obscor_clim.lst,obscor_std.lst)
mpd_cor_rms = rmse(mpd_cor.tas,obscor.lst)

/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs/env3/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/scratch/brussel/vo/000/bvo00012/vsc10314/miniconda/envs

In [10]:
dif_rms = mpd_rms - mnlut_rms
dif_kge = mpd_kge - mnlut_kge
difcor_rms = mpd_cor_rms - mnlut_cor_rms
difcor_kge = mpd_cor_kge - mnlut_cor_kge

In [11]:
dif_rms = dif_rms.to_dataset()
dif_kge = dif_kge.to_dataset()
difcor_rms = difcor_rms.to_dataset()
difcor_kge = difcor_kge.to_dataset()

In [13]:
dif_rms.to_netcdf('review_files/rmse_lst_dif_'+model+'.nc')
dif_kge.to_netcdf('review_files/kge_lst_dif_'+model+'.nc')
difcor_rms.to_netcdf('review_files/rmse_lst_difcor_'+model+'.nc')
difcor_kge.to_netcdf('review_files/kge_lst_difcor_'+model+'.nc')

In [16]:
mnlut_rms = mnlut_rms.to_dataset()
mnlut_kge=mnlut_kge.to_dataset()
mnlut_cor_rms = mnlut_cor_rms.to_dataset()
mnlut_cor_kge=mnlut_cor_kge.to_dataset()

In [17]:
mpd_rms = mpd_rms.to_dataset()
mpd_kge=mpd_kge.to_dataset()
mpd_cor_rms = mpd_cor_rms.to_dataset()
mpd_cor_kge=mpd_cor_kge.to_dataset()

In [21]:
mnlut_rms.to_netcdf('review_files/mnlut_lst_rms_'+model+'_2new.nc')
mnlut_kge.to_netcdf('review_files/mnlut_lst_kge_'+model+'_2new.nc')
mnlut_cor_rms.to_netcdf('review_files/mnlut_lst_cor_rms_'+model+'_2new.nc')
mnlut_cor_kge.to_netcdf('review_files/mnlut_lst_cor_kge_'+model+'_2new.nc')

In [19]:
mpd_rms.to_netcdf('review_files/mpd_lst_rms_'+model+'_2new.nc')
mpd_kge.to_netcdf('review_files/mpd_lst_kge_'+model+'_2new.nc')
mpd_cor_rms.to_netcdf('review_files/mpd_lst_cor_rms_'+model+'_2new.nc')
mpd_cor_kge.to_netcdf('review_files/mpd_lst_cor_kge_'+model+'_2new.nc')